# Práctica 4

**Nombre:** Anahí Barajas Pérez  
**e-mail:** anahi.barajas2412@alumnos.udg.mx

## MODULES

In [ ]:
import panel as pn
import panel.widgets as pnw
import holoviews as hv
#from holoviews import opts
pn.extension(sizing_mode = 'stretch_width')
hv.extension('plotly')

import pandas as pd
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from pymunk import Vec2d
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable


## FUNCTIONS

In [ ]:
def BM_2d(n_steps=1000,speed=6,s_x_pos=0,s_y_pos=0):
    '''
    Arguments:
        n_steps:
        speed:
        s_pos:
    Returns:
        BM_2d_df:
      '''
    #Init Vector
    velocity = Vec2d(speed,0)

    BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df],ignore_index=True)

    for i in range(n_steps - 1):
        turn_angle = np.random.uniform(low=-np.pi,high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index = True)

    #Return dataframe
    return BM_2d_df

In [ ]:
def CRW_2d(n_steps=1000,speed=6,s_x_pos=0,s_y_pos=0,crw_exponent=0.3):      
    '''
    Arguments:
        n_steps:
        speed:
        s_pos:
        crw_exponent:
    Returns:
        CRW_2d_df:
    '''
    # Init velocity vector 
    velocity = Vec2d(speed,0)
    
    # Generate turn angles 
    rv = wrapcauchy.rvs(crw_exponent, size=n_steps)

    #Init DataFrame
    CRW_2d_df = pd.DataFrame([{'x_pos':s_x_pos, 'y_pos':s_y_pos }])

    for i in range(n_steps-1):    
        velocity = velocity.rotated(rv[i])
        CRW_2d_df = pd.concat([CRW_2d_df,pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i]+velocity.x, 'y_pos': CRW_2d_df.y_pos[i]+velocity.y}])], ignore_index = True)
    
    #Return dataframe
    return CRW_2d_df

In [ ]:
def LF_2d(n_steps=1000,speed=6,s_x_pos=0,s_y_pos=0,crw_exponent=0.3,levy_exponent = 1.9,beta = 0):

    # Init vectors
    wrapcauchy_rvs = wrapcauchy.rvs(crw_exponent, size=n_steps)
    levy_stable_rvs = levy_stable.rvs(levy_exponent,beta,size=n_steps)
    velocity = Vec2d(speed,0)
    turn_angle = 0

    # Init Data Frame
    LF_2d_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos':s_y_pos}]) 

    for i in range(n_steps-1):

        sl = levy_stable_rvs[i]

        x_pos = sl * np.cos(turn_angle)
        y_pos = sl * np.sin(turn_angle)

        turn_angle = wrapcauchy_rvs[i]
        
        velocity = Vec2d(x_pos,y_pos)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': LF_2d_df.x_pos[i]+velocity.x, 'y_pos': LF_2d_df.y_pos[i]+velocity.y}])

        LF_2d_df = pd.concat([LF_2d_df,temp_df], ignore_index = True)  
    
    return LF_2d_df

In [ ]:
def pl_metric(traj_df):
    '''
    Arguments:
        traj_df: Trajectory dataframe
    Return:
        metric_df: Metric dataframe
    '''
    # BM speed = 3
    vec_aux = np.empty(0)    
 
    for i in range(1,len(traj_df)):
        dis = np.sqrt(np.square((traj_df.x_pos[i] - traj_df.x_pos[i-1])) + np.square((traj_df.y_pos[i] - traj_df.y_pos[i-1])))
        vec_aux = np.append(vec_aux,dis)

    pl_cumsum = np.cumsum(vec_aux)
    metric_df = pd.DataFrame(pl_cumsum, columns=['M']) # Saving metrics in DataFrame
        
           
    #Return dataframe
    return metric_df

In [ ]:
def msd_metric(traj_df):
    '''
    Arguments:
        traj_df: Trajectory dataframe
    Return:
        metric_df: Metric dataframe
    '''    
    vec_aux = np.empty(0)
    
    for n in range(1,len(traj_df)):
        msd = np.mean(np.array([np.square(np.sqrt(np.square((traj_df.x_pos[j] - traj_df.x_pos[j-n])) + np.square((traj_df.y_pos[j] - traj_df.y_pos[j-n])))) for j in range(n,len(traj_df))]))
        vec_aux = np.append(vec_aux,msd)

    metric_df = pd.DataFrame(vec_aux, columns=['M'])    
    
    return metric_df

In [ ]:
rTrajTyp = pnw.RadioButtonGroup(name='Trajectorie Type', options=['BM', 'CRW', 'LF'], value='BM', button_type='primary')
slStep = pnw.IntSlider(name='Number of steps', value = 1000, start=1000, end=10000, step = 1000)
xInput = pnw.IntInput(name = 'X intial position', value = 0, start=0, end=200)
yInput = pnw.IntInput(name = 'Y intial position', value = 0, start=0, end=200)
slSpeed = pnw.IntSlider(name='Speed', value = 3, start = 1, end = 10, sizing_mode='stretch_width')
selMetric = pnw.Select(name='Metric Type', options=['MSD','PL'], value = 'PL', sizing_mode='stretch_width')
slCRW = pnw.FloatSlider(name='CRW Exponent', value = 0.3, start = 0, end = 1, sizing_mode='stretch_width')
slLevyA = pnw.FloatSlider(name='Lévy α', value = 1.9, start = 0, end = 2, sizing_mode='stretch_width')
slLevyB = pnw.FloatSlider(name='Lévy β', value = 0, start = -1, end = 1, sizing_mode='stretch_width')

@pn.depends(rTrajTyp)
def dCRWSlider(rTrajTyp):
    if rTrajTyp != 'BM':
        slCRW.visible = True
    else:
        slCRW.visible = False

    return slCRW

@pn.depends(rTrajTyp)
def dLevyASlider(rTrajTyp):
    return slLevyA

@pn.depends(rTrajTyp)
def dLevySlider(rTrajTyp):
    n=pn.GridBox(dLevyASlider,slLevyB,ncols=1)
    if rTrajTyp == 'LF':
        n.visible = True
    else:
        n.visible = False
    return n


@pn.depends(slStep=slStep, slSpeed=slSpeed, xInput=xInput, yInput=yInput)
def dSpecs(slStep, slSpeed, xInput, yInput):
    text = 'Steps: **'+str(slStep)+'** Speed: **'+str(slSpeed)+'** Initial Position: ['+str(xInput)+','+str(yInput)+']'
    return pn.pane.Markdown(text)

@pn.depends(rTrajTyp=rTrajTyp,slCRW=slCRW)
def dSpecsCRW(rTrajTyp,slCRW):
    if rTrajTyp != 'BM':
        text = 'CRW exponent: **'+str(slCRW)+'**'
    else:
        text = ''
    return pn.pane.Markdown(text)

@pn.depends(rTrajTyp=rTrajTyp,slLevyA=slLevyA,slLevyB=slLevyB)
def dSpecsLF(rTrajTyp,slLevyA,slLevyB):
    if rTrajTyp == 'LF':
        text = 'Alpha: **'+str(slLevyA)+'** Beta: **'+str(round(slLevyB,2))+'**'
    else:
        text = ''
    return pn.pane.Markdown(text)

@pn.depends(rTrajTyp=rTrajTyp,slStep=slStep, slSpeed=slSpeed, xInput=xInput, yInput=yInput, slCRW=slCRW,selMetric=selMetric)
def dTitle(rTrajTyp, slStep, slSpeed, xInput, yInput, slCRW,selMetric):
    
    if rTrajTyp == 'BM':   
        tTyp = 'Brownian Motion'
    elif rTrajTyp == 'CRW':
        tTyp = 'Correlated Random Walk'
    elif rTrajTyp == 'LF':
        tTyp = 'Lévy Flight'
    
    text = '##### **'+tTyp+'**'
            
    return pn.pane.Markdown(text, sizing_mode='stretch_width')


In [ ]:
@pn.depends(rTrajTyp=rTrajTyp,slStep=slStep, slSpeed=slSpeed, xInput=xInput, yInput=yInput, slCRW=slCRW,selMetric=selMetric,slLevyA=slLevyA,slLevyB=slLevyB)
def plot_rw(rTrajTyp, slStep, slSpeed, xInput, yInput, slCRW,selMetric,slLevyA,slLevyB):
        
    if rTrajTyp == 'BM':   
        rw_df = BM_2d(n_steps=slStep, speed=slSpeed, s_x_pos=xInput, s_y_pos=yInput)
    elif rTrajTyp == 'CRW':
        rw_df = CRW_2d(n_steps=slStep, speed=slSpeed, s_x_pos=xInput, s_y_pos=yInput, crw_exponent=slCRW)  
    elif rTrajTyp == 'LF':
        rw_df = LF_2d(n_steps=slStep, speed=slSpeed, s_x_pos=xInput, s_y_pos=yInput, crw_exponent=slCRW, levy_exponent=slLevyA, beta=slLevyB) 
            

    metricTp = 'Path Lenght' if selMetric == 'PL' else 'Mean Squared Displacement'
    
    column_titles=[f'{rTrajTyp} 3D Trajectory',f'{metricTp}']
    
    fig = make_subplots(cols=2,specs=[[{'type': 'scene'}, {'type': 'xy'}]]
                        ,subplot_titles =column_titles
                        ,column_widths=[.6,.4])
    
    
    pos_in = pd.DataFrame([{'x':rw_df.x_pos[0], 'y':rw_df.y_pos[0], 'z':rw_df.index[0]}])
    pos_in = pd.concat([pos_in,pd.DataFrame([{'x':rw_df['x_pos'].iloc[-1], 'y':rw_df['y_pos'].iloc[-1], 'z':rw_df.index[-1]}])], ignore_index = True)  
    
    fig.add_trace(
        
        go.Scatter3d(
            x=rw_df.x_pos,
            y=rw_df.y_pos,
            z=rw_df.index,
            line_width=2,
            line_color='#06768d',
            opacity=.6,
            mode = 'lines'
        ),row=1,col=1)
    
    fig.add_trace(        
        go.Scatter3d(
            x=pos_in.x,
            y=pos_in.y,
            z=pos_in.z,
            marker_size=4,
            mode = 'markers',
            marker_color='red'
        ),row=1,col=1)
    
    
    metrics_df = pl_metric(rw_df) if selMetric == 'PL' else msd_metric(rw_df)

    fig.add_trace(
        go.Scatter(
            x = np.arange(len(metrics_df.M)),
            y = metrics_df.M,
            line_width = 2,
            line_color = '#cb2424',
            mode = 'lines',
            showlegend = True
        ), row=1,col=2)   

    fig.update_layout(autosize=True,showlegend=False,margin=dict(l=26, r=26, t=26, b=26))
     #Return plot figure
    return fig

In [ ]:
bootstrap = pn.template.BootstrapTemplate(title='Random Walk Metrics: Path Length/MSD'
                                          ,sidebar_width = 220
                                          ,header_background='#00504C')

bootstrap.sidebar.append(rTrajTyp)
bootstrap.sidebar.append(slStep)
bootstrap.sidebar.append(xInput)
bootstrap.sidebar.append(yInput)
bootstrap.sidebar.append(slSpeed)
bootstrap.sidebar.append(dCRWSlider)
bootstrap.sidebar.append(dLevySlider)
bootstrap.sidebar.append(selMetric)

bootstrap.main.append(
    pn.Column(
        pn.Row(dTitle, sizing_mode='stretch_width'),    
        pn.Row(plot_rw, sizing_mode='stretch_width',css_classes=['panel-widget-box']),
        pn.Row(dSpecs,dSpecsCRW,dSpecsLF, sizing_mode='stretch_width'), 
    )
)

bootstrap.show();